<a href="https://colab.research.google.com/github/NRmethun/ML-DL-Notebook/blob/main/Mobile_net_attempt_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np

In [ ]:
os.chdir('/content/drive/MyDrive/audio-features')
print(os.getcwd())

/content/drive/MyDrive/audio-features


In [ ]:
features = np.load('train_feature_x_batch_1.npy', allow_pickle=True)
labels = np.load('train_label_y_batch_1.npy', allow_pickle=True)
for i in range(2,31):
  file_name_x =  f'train_feature_x_batch_{i}.npy'
  file_name_y = f'train_label_y_batch_{i}.npy'
  # print("------------------------------------------------")
  # print(file_name_x)
  # print(file_name_y)
  features_new =  np.load(file_name_x, allow_pickle=True)
  lebels_new = np.load(file_name_y, allow_pickle=True)
  features = np.concatenate((features, features_new), axis=0)
  labels = np.concatenate((labels,lebels_new))
  print(i)
  # print("------------------------------------------------")
  del features_new
  del lebels_new

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [ ]:
features.shape

(13500, 128, 300, 3)

In [ ]:
features_test = np.load('train_feature_x_batch_122.npy', allow_pickle=True)
labels_test = np.load('train_label_y_batch_122.npy', allow_pickle=True)
for i in range(122,127):
  file_name_x =  f'train_feature_x_batch_{i}.npy'
  file_name_y = f'train_label_y_batch_{i}.npy'
  print(file_name_x)
  print(file_name_y)
  features_new =  np.load(file_name_x, allow_pickle=True)
  lebels_new = np.load(file_name_y, allow_pickle=True)
  features_test = np.concatenate((features_test, features_new), axis=0)
  labels_test = np.concatenate((labels_test,lebels_new))
  print(i)
  del features_new
  del lebels_new

train_feature_x_batch_122.npy
train_label_y_batch_122.npy
122
train_feature_x_batch_123.npy
train_label_y_batch_123.npy
123
train_feature_x_batch_124.npy
train_label_y_batch_124.npy
124
train_feature_x_batch_125.npy
train_label_y_batch_125.npy
125
train_feature_x_batch_126.npy
train_label_y_batch_126.npy
126


In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Example list of categories


# Create an instance of the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Reshape the list into a 2D array
y_train = np.array(labels).reshape(-1, 1)
y_test = np.array(labels_test).reshape(-1, 1)

# Fit and transform the data using the encoder
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
# Print the one-hot encoded array
# print(y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
y_train.shape
y_test[11]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D ,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
mobilnet = VGG16(input_shape=(128,300,3),
                 weights='imagenet',
                 #weights=None ,
                 include_top=False)



for layer in mobilnet.layers:
  layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
last_output = mobilnet.output
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)

# Add a final sigmoid layer for classification
x = layers.Dense  (45, activation='softmax')(x)

model = Model( mobilnet.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('saved_mode_vgg19_90.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto')

In [ ]:
history = model.fit(features, y_train, epochs=300,
                    batch_size=32,
                       callbacks=[checkpoint],
                       validation_data= (features_test, y_test))

Epoch 1/300
422/422 [==============================] - ETA: 0s - loss: 3.8140 - accuracy: 0.0210
Epoch 1: val_accuracy improved from -inf to 0.02348, saving model to saved_mode_vgg19_90.h5
422/422 [==============================] - 72s 137ms/step - loss: 3.8140 - accuracy: 0.0210 - val_loss: 3.8067 - val_accuracy: 0.0235
Epoch 2/300
422/422 [==============================] - ETA: 0s - loss: 3.8083 - accuracy: 0.0224

KeyboardInterrupt: ignored

In [ ]:
history_dict=history.history
loss_values=history_dict['loss']
acc_values=history_dict['accuracy']
val_loss_values = history_dict['val_loss']
val_acc_values=history_dict['val_accuracy']
epochs=range(1,301)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.plot(epochs,loss_values,'bo',label='Training Loss')
ax1.plot(epochs,val_loss_values,'orange', label='Validation Loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(epochs,acc_values,'bo', label='Training accuracy')
ax2.plot(epochs,val_acc_values,'orange',label='Validation accuracy')
ax2.set_title('Training and validation accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
model.load_weights('saved_mode_vgg19_90.h5')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[INFO] Calculating model accuracy
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-eb032969c1a1>", line 2, in <cell line: 2>
    model.load_weights('saved_mode_vgg19_90.h5')
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 567, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 231, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 106, in h5py.h5f.open
OSError: [

In [ ]:
Y_pred = model.predict(features_test)


In [ ]:
Y_pred_for_conf = np.argmax(Y_pred,axis =1)
y_test_for_conf = np.argmax(y_test, axis =1)

In [ ]:
all_class = np.unique(labels_test)
all_class

In [ ]:
classes = ['1', '10', '11', '12', '14', '15', '16', '17', '18', '19', '2',
       '25', '26', '27', '3', '4', '47', '48', '5', '55', '56', '57',
       '58', '6', '61', '70', '8', '80', '81', '82', '83', '84', '85',
       '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95',
       '98']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test_for_conf, Y_pred_for_conf, target_names=classes))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming you have ground truth labels and predicted labels stored in variables
# y_true: Ground truth labels
# y_pred: Predicted labels

# Compute confusion matrix
cm = confusion_matrix(y_test_for_conf, Y_pred_for_conf)

# Create a list of class labels


# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

# Normalize the confusion matrix if desired
# cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Format and display the actual values in the confusion matrix
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()


In [ ]:
print(cm)

In [ ]:
np.savetxt("/content/drive/MyDrive/outputs/confusion_matrix_attempt3_vgg19_90.csv", cm, delimiter=",")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-32a000d8fa72>", line 1, in <cell line: 1>
    np.savetxt("/content/drive/MyDrive/outputs/confusion_matrix_attempt3_vgg19_90.csv", cm, delimiter=",")
  File "<__array_function__ internals>", line 180, in savetxt
  File "/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py", line 1524, in savetxt
    fh = np.lib._datasource.open(fname, 'wt', encoding=encoding)
  File "/usr/local/lib/python3.10/dist-packages/numpy/lib/_datasource.py", line 192, in open
    ds = DataSource(destpath)
  File "/usr/local/lib/python3.10/dist-packages/numpy/lib/_datasource.py", line 249, in __init__
    self._destpath = os.path.abspath(destpath)
  File "/usr/lib/python3.10/posixpath.py", line 384, in abspath
    cwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

D

In [ ]:
from keras.applications.mobilenet import MobileNet

In [ ]:
mobilnet = MobileNet(input_shape=(128,300,3),
                 weights='imagenet',
                 #weights=None ,
                 include_top=False)



for layer in mobilnet.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)

# Add a final sigmoid layer for classification
x = layers.Dense  (45, activation='softmax')(x)

model = Model( mobilnet.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-24a4295de430>", line 16, in <cell line: 16>
    model = Model( mobilnet.input, x)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py", line 167, in __init__
    self._init_graph_network(inputs, outputs)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py", line 266, in _init_graph_network
    nodes, nodes_by_depth, layers, _ = _map_graph_network(
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/functi

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-24a4295de430>", line 16, in <cell line: 16>
    model = Model( mobilnet.input, x)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py", line 167, in __init__
    self._init_graph_network(inputs, outputs)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py", line 266, in _init_graph_network
    nodes, nodes_by_depth, layers, _ = _map_graph_network(
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/functi

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('saved_mode_mobile_net_90.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto')

In [ ]:
history = model.fit(features, y_train, epochs=300,
                    batch_size=32,
                       callbacks=[checkpoint],
                       validation_data= (features_test, y_test))

Epoch 1/300
1266/1266 [==============================] - ETA: 0s - loss: 0.2669 - accuracy: 0.9199
Epoch 1: val_accuracy improved from -inf to 0.97198, saving model to saved_mode_mobile_net_90.h5


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-f86976be6728>", line 1, in <cell line: 1>
    history = model.fit(features, y_train, epochs=300,
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 567, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 237, in make_fid
    fid = h5f.create(name, h5f.ACC_TRUNC, fapl=fapl, fcpl=fcpl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 126, in h5py.h5f.create
OSError: [Errno

In [ ]:
history_dict=history.history
loss_values=history_dict['loss']
acc_values=history_dict['accuracy']
val_loss_values = history_dict['val_loss']
val_acc_values=history_dict['val_accuracy']
epochs=range(1,301)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.plot(epochs,loss_values,'bo',label='Training Loss')
ax1.plot(epochs,val_loss_values,'orange', label='Validation Loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(epochs,acc_values,'bo', label='Training accuracy')
ax2.plot(epochs,val_acc_values,'orange',label='Validation accuracy')
ax2.set_title('Training and validation accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
model.load_weights('saved_mode_mobile_net_90.h5')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[INFO] Calculating model accuracy
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-4c3afbef3abb>", line 2, in <cell line: 2>
    model.load_weights('saved_mode_mobile_net_90.h5')
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 567, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 231, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 106, in h5py.h5f.open
OSErr

In [ ]:
Y_pred = model.predict(features_test)


In [ ]:
Y_pred_for_conf = np.argmax(Y_pred,axis =1)
y_test_for_conf = np.argmax(y_test, axis =1)

In [ ]:
all_class = np.unique(labels_test)
all_class

In [ ]:
classes = ['1', '10', '11', '12', '14', '15', '16', '17', '18', '19', '2',
       '25', '26', '27', '3', '4', '47', '48', '5', '55', '56', '57',
       '58', '6', '61', '70', '8', '80', '81', '82', '83', '84', '85',
       '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95',
       '98']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test_for_conf, Y_pred_for_conf, target_names=classes))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming you have ground truth labels and predicted labels stored in variables
# y_true: Ground truth labels
# y_pred: Predicted labels

# Compute confusion matrix
cm = confusion_matrix(y_test_for_conf, Y_pred_for_conf)

# Create a list of class labels


# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

# Normalize the confusion matrix if desired
# cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Format and display the actual values in the confusion matrix
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()


In [ ]:
print(cm)

In [ ]:
np.savetxt("/content/drive/MyDrive/outputs/confusion_matrix_attempt5_mobile_net_90.csv", cm, delimiter=",")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-9ad83a543ae4>", line 1, in <cell line: 1>
    np.savetxt("/content/drive/MyDrive/outputs/confusion_matrix_attempt5_mobile_net_90.csv", cm, delimiter=",")
  File "<__array_function__ internals>", line 180, in savetxt
  File "/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py", line 1524, in savetxt
    fh = np.lib._datasource.open(fname, 'wt', encoding=encoding)
  File "/usr/local/lib/python3.10/dist-packages/numpy/lib/_datasource.py", line 192, in open
    ds = DataSource(destpath)
  File "/usr/local/lib/python3.10/dist-packages/numpy/lib/_datasource.py", line 249, in __init__
    self._destpath = os.path.abspath(destpath)
  File "/usr/lib/python3.10/posixpath.py", line 384, in abspath
    cwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connect

In [ ]:
#inception

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (128,300,3),
                                include_top = False,
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)

# Add a final sigmoid layer for classification
x = layers.Dense  (45, activation='softmax')(x)

model = Model( mobilnet.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('saved_mode_inception_v3_90.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto')

In [ ]:
history = model.fit(features, y_train, epochs=300,
                    batch_size=32,
                       callbacks=[checkpoint],
                       validation_data= (features_test, y_test))

In [ ]:
history_dict=history.history
loss_values=history_dict['loss']
acc_values=history_dict['accuracy']
val_loss_values = history_dict['val_loss']
val_acc_values=history_dict['val_accuracy']
epochs=range(1,301)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.plot(epochs,loss_values,'bo',label='Training Loss')
ax1.plot(epochs,val_loss_values,'orange', label='Validation Loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(epochs,acc_values,'bo', label='Training accuracy')
ax2.plot(epochs,val_acc_values,'orange',label='Validation accuracy')
ax2.set_title('Training and validation accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
model.load_weights('saved_mode_inception_v3_90.h5')

In [ ]:
Y_pred = model.predict(features_test)


In [ ]:
Y_pred_for_conf = np.argmax(Y_pred,axis =1)
y_test_for_conf = np.argmax(y_test, axis =1)

In [ ]:
all_class = np.unique(labels_test)
all_class

In [ ]:
classes = ['1', '10', '11', '12', '14', '15', '16', '17', '18', '19', '2',
       '25', '26', '27', '3', '4', '47', '48', '5', '55', '56', '57',
       '58', '6', '61', '70', '8', '80', '81', '82', '83', '84', '85',
       '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95',
       '98']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test_for_conf, Y_pred_for_conf, target_names=classes))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming you have ground truth labels and predicted labels stored in variables
# y_true: Ground truth labels
# y_pred: Predicted labels

# Compute confusion matrix
cm = confusion_matrix(y_test_for_conf, Y_pred_for_conf)

# Create a list of class labels


# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

# Normalize the confusion matrix if desired
# cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Format and display the actual values in the confusion matrix
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()


In [ ]:
print(cm)

In [ ]:
np.savetxt("/content/drive/MyDrive/outputs/confusion_matrix_attempt5_inception_v3_90.csv", cm, delimiter=",")